# If you like my kernel, do upvote it :)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import copy

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets_scored = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_targets_nonscored = pd.read_csv('/kaggle/input/lish-moa/train_targets_nonscored.csv')
sample_submission = pd.read_csv('/kaggle/input/lish-moa/sample_submission.csv')

In [ ]:
train_features.head()

In [ ]:
train_targets_scored.head()

In [ ]:
print("# of common columns : ",len(set(sample_submission.columns.values.tolist()) & set(train_targets_scored.columns.values.tolist())))
print("# of columns in sample submission : ", len(sample_submission.columns.values.tolist()))

In [ ]:
print(train_features.shape)
print(train_targets_scored.shape)

### We have 23814 training examples with 876 features and 207 columns to predict on [MultiClass]

In [ ]:
train_features.describe()

In [ ]:
train_targets_scored.describe()

In [ ]:
GENE = [col for col in train_features.columns if col.startswith('g-')]
len(GENE)

In [ ]:
CELL = [col for col in train_features.columns if col.startswith('c-')]
len(CELL)

In [ ]:
plt.figure(figsize=(16,16))
sns.set_style("whitegrid")

gene_choice = np.random.choice(len(GENE), 16)
for i, col in enumerate(gene_choice):
    plt.subplot(4, 4, i+1)
    plt.hist(train_features.loc[:, GENE[col]],bins=100, color='orange')
    plt.title(GENE[col])

In [ ]:
plt.figure(figsize=(16,16))
sns.set_style("whitegrid")

cell_choice = np.random.choice(len(CELL), 16)
for i, col in enumerate(cell_choice):
    plt.subplot(4, 4, i+1)
    plt.hist(train_features.loc[:, CELL[col]],bins=100, color='green')
    plt.title(CELL[col])

In [ ]:
train = pd.merge(train_features, train_targets_scored, on='sig_id')

In [ ]:
train.head()

In [ ]:
target_columns = train_targets_scored.columns

In [ ]:
sns.distplot(train_targets_scored[[col for col in train_targets_scored.columns if col not in ['sig_id']]\
                    ].sum(axis=1))

In [ ]:
train_targets_scored[\
                     [col for col in train_targets_scored.columns if col not in ['sig_id']]\
                    ].sum(axis=1)\
                    .unique()

### Observation:
1. In some cases,we have multiple labels per datapoint
2. Some of the datapoints even have 0/206 labels.

In [ ]:
train.head()

In [ ]:
# train.values[0][876:]

>  ## Each sig_id can have multiple labels, so here I'll be breaking up my dataframe such that each row has a single label
 

In [ ]:
X_train = []
X_train_columns = train.columns

for v in train.values:
    info = v[:876]
    binary = v[876:]
    index = [k for k, i in enumerate(binary) if i==1]
    
    for i in index:
        for k in range(len(binary)):
            if k==i:
#                 binary_transformed = list(copy.copy(binary))
#                 binary_transformed[i] = 0
                X_train.append(list(info) + [X_train_columns[876+k]])

X_train = pd.DataFrame(X_train, columns=train_features.columns.tolist() + ['pred'])
# X_train.columns = X_train_columns.tolist() + ['pred']

In [ ]:
X_train.head()

In [ ]:
X_train.shape

### Number of training examples reduced, as some of the datapoints had no labels at all out of 206 labels

In [ ]:
X_test = test_features
y_train = X_train[['pred']]

In [ ]:
y_train.head()

In [ ]:
len(y_train['pred'].unique().tolist())

In [ ]:
X_train

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

X_test['pred'] = -1
data = X_train.append(X_test)

for col in ['cp_type','cp_time','cp_dose']:
    data.loc[:, col] = le.fit_transform(data[col])
    
X_train = data[:X_train.shape[0]]
X_test = data[-X_test.shape[0]:]

In [ ]:
y_train

In [ ]:
mapping = {v:k for k,v in enumerate(sample_submission.iloc[:,1:].columns.tolist())}
# inverse_mapping = {k:v for k,v in enumerate(sample_submission.iloc[:,1:].columns.tolist())}

In [ ]:
y_train['pred'] = y_train['pred'].map(mapping)
y_train

In [ ]:
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

In [ ]:
cat_features = ['cp_type','cp_time','cp_dose']

model = CatBoostClassifier(task_type='GPU')

model.fit(X_train.drop(columns=['sig_id','pred']),\
          y_train,\
          verbose=True,\
          cat_features=cat_features)


In [ ]:
prediction = model.predict_proba(X_test.drop(columns=['sig_id'], axis=1))

In [ ]:
sample_submission.iloc[:, 1:] = prediction

In [ ]:
sample_submission.to_csv('submission.csv',index=False)